In [192]:
from sklearn.datasets import fetch_openml
import pandas as pd
import torch
import torch.nn as nn  # LinearRegressoion 기능의 클래스 Linear
import torch.nn.functional as F
import torch.optim as optim
from sklearn.preprocessing import OneHotEncoder

In [193]:
fashion_data = fetch_openml(name='Fashion-MNIST', parser='auto', as_frame=True)

In [194]:
# 정규화 ==> 피쳐 정규화 : 784개는 색상값 즉, 0 ~ 255 범위의 값
feature = fashion_data.data / 255.  # 실수로 나누기 ! => 정수/실수 = 실수 => 형변환
target = fashion_data.target

In [195]:
feature.shape, target.shape

((70000, 784), (70000,))

In [196]:
target = target.astype('float')

In [197]:
target.shape

(70000,)

In [198]:
xtrain = feature[:60000]
ytrain = target[:60000][:]

xtest = feature[60000:]
ytest = target[60000:][:]
ytrain.shape, ytest.shape

((60000,), (10000,))

In [199]:
# 텐서로 바꾸기
xtrain = torch.tensor(xtrain.values, dtype=torch.float32)
xtest = torch.tensor(xtest.values, dtype=torch.float32)

In [200]:
xtrain.shape, xtrain.ndim, xtest.shape

(torch.Size([60000, 784]), 2, torch.Size([10000, 784]))

In [201]:
ytrain = torch.tensor(ytrain.values, dtype=torch.float32).unsqueeze(dim = 1)
ytest = torch.tensor(ytest.values, dtype=torch.float32).unsqueeze(dim = 1)

In [202]:
ytrain.shape, ytrain.ndim, ytest.shape, ytest.ndim

(torch.Size([60000, 1]), 2, torch.Size([10000, 1]), 2)

In [203]:
# one-hot encoding
y_one_hot = torch.zeros(60000, 10)
y_one_hot1 = torch.zeros(10000,10)
# torch.scatter_(input, dim, index, src)
# input: 값이 할당될 대상 텐서 - y_one_hot
# dim: 값을 할당할 차원
# index: 값이 할당될 위치를 나타내는 텐서
# src: input 텐서에 할당될 값
ytrain = y_one_hot.scatter(1, ytrain.long(), 1)
ytest = y_one_hot1.scatter(1,ytest.long(),1)

In [212]:
ytrain

tensor([[0., 0., 0.,  ..., 0., 0., 1.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [204]:
class myModel(nn.Module): # nn.Module : 부모클래스
    
    # 모델 구조설정
    def __init__(self, in_,out_):
        super().__init__()
        self.layer = nn.Linear(in_,out_)
        self.af = nn.Softmax(dim=1)
        
    # 순방향 학습 진행 콜백 함수
    def forward(self, x): # 입력데이터 : x
        y = self.layer(x)
        y = self.af(y)

        return y

In [205]:
model = myModel(784,10)

In [225]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.5)

In [226]:
epochs = 1000
min_loss = 100.0
for epoch in range(1 + epochs):
    ypre = model(xtrain)    
    loss = F.binary_cross_entropy(ypre, ytrain)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    with torch.no_grad():
        ypre_t = model(xtest)
        test_loss = F.binary_cross_entropy(ypre_t, ytest)
        assert test_loss.requires_grad == False
    
    
    if epoch % 10 == 0:

        
        accuracy = sum(ypre.argmax(dim = 1) == ytrain.argmax(dim = 1)) / ytrain.shape[0]

        print(f'Epoch {epoch:4d}/{epochs} Cost: {loss.item():.2f} Accuracy {accuracy * 100:2.2f}%')
        if loss < min_loss:
            min_loss = loss
        else:
            print('학습 종료')
            break


Epoch    0/1000 Cost: 0.09 Accuracy 83.55%
Epoch   10/1000 Cost: 0.09 Accuracy 83.57%
Epoch   20/1000 Cost: 0.09 Accuracy 83.58%
Epoch   30/1000 Cost: 0.09 Accuracy 83.60%
Epoch   40/1000 Cost: 0.09 Accuracy 83.62%
Epoch   50/1000 Cost: 0.09 Accuracy 83.62%
Epoch   60/1000 Cost: 0.09 Accuracy 83.64%
Epoch   70/1000 Cost: 0.08 Accuracy 83.67%
Epoch   80/1000 Cost: 0.08 Accuracy 83.67%
Epoch   90/1000 Cost: 0.08 Accuracy 83.68%
Epoch  100/1000 Cost: 0.08 Accuracy 83.71%
Epoch  110/1000 Cost: 0.08 Accuracy 83.72%
Epoch  120/1000 Cost: 0.08 Accuracy 83.74%
Epoch  130/1000 Cost: 0.08 Accuracy 83.75%
Epoch  140/1000 Cost: 0.08 Accuracy 83.77%
Epoch  150/1000 Cost: 0.08 Accuracy 83.78%
Epoch  160/1000 Cost: 0.08 Accuracy 83.80%
Epoch  170/1000 Cost: 0.08 Accuracy 83.83%
Epoch  180/1000 Cost: 0.08 Accuracy 83.84%
Epoch  190/1000 Cost: 0.08 Accuracy 83.85%
Epoch  200/1000 Cost: 0.08 Accuracy 83.86%
Epoch  210/1000 Cost: 0.08 Accuracy 83.88%
Epoch  220/1000 Cost: 0.08 Accuracy 83.89%
Epoch  230/

In [260]:
with torch.no_grad():
    ypre = model(xtest)
    test_loss = F.binary_cross_entropy(ypre, ytest)
    assert test_loss.requires_grad == False

In [241]:
xtest[1].shape,xtest[1].ndim
test = xtest[1].unsqueeze(dim = 0)

In [242]:
ypre = model(test)  # 1일 확신도가 0.999757

In [254]:
ytest[1].shape, ytest[1].ndim
ans_test = ytest[1].unsqueeze(dim = 0)

In [259]:
print(f' 예측값 : {ypre.argmax(dim = 1).item()} / 실제값 : {ans_test.argmax(dim = 1).item()}')

 예측값 : 2 / 실제값 : 2
